# Chapter 6: Advanced Spark Programming (Python)

In [49]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder.appName("Advanced-Spark-Programming").master("local[*]").getOrCreate()
val sc = spark.sparkContext

spark = org.apache.spark.sql.SparkSession@ed87fc2
sc = org.apache.spark.SparkContext@244ac242


Spark local-1533235988904: Some(http://d2eb42893bfa:4041)

## Accumulators

In [50]:
val acc_5 = sc.accumulator(0)

acc_5 = 0


0

In [51]:
val numRdd = sc.parallelize(List(1,2,5,5,3,6,9,6,5,9))

numRdd = ParallelCollectionRDD[9] at parallelize at <console>:30


ParallelCollectionRDD[9] at parallelize at <console>:30

In [52]:
numRdd.glom().collect()

[[1, 2], [5, 5, 3], [6, 9], [6, 5, 9]]

In [53]:
object Stack {
    
    def overFlow(): Unit = {
        
        val acc_5 = sc.accumulator(0)
        
        val numRdd = sc.parallelize(List(1,2,5,5,3,6,9,6,5,9))
        numRdd.map(num => {if(num == 5){acc_5+=1}}).collect()
        println(acc_5)    
    }
}

defined object Stack


In [54]:
Stack.overFlow

3
